In [1]:
# *******************************************************************
#
# Author : Thanh Nguyen, 2018
# Email  : sthanhng@gmail.com
# Github : https://github.com/sthanhng
#
# A collection of OpenCV libraries
#
# Description : helpers.py
#
# *******************************************************************


import numpy as np
import cv2
import os

valid_exts = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')


def rotate(img, angle, center=None, scale=1.0):
    # get the dimensions of the image
    (height, width) = img.shape[:2]

    # if the center is None, initialize it as the center of the image
    if center == None:
        center = (width / 2, height / 2)

    # the rotation
    rota_matrix = cv2.getRotationMatrix2D(center, angle, scale)
    rotated_img = cv2.warpAffine(img, rota_matrix, (width, height))

    # return the rotated image
    return rotated_img


def resize(image, resized_width=None, resized_height=None, inter=cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # get the image size
    (height, width) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if resized_width is None and resized_height is None:
        return image

    if resized_width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        ratio = resized_height / float(height)
        dim = (int(width * ratio), resized_height)
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        ratio = resized_width / float(width)
        dim = (resized_width, int(height * ratio))

    # resize the image
    resized = cv2.resize(image, dim, interpolation=inter)

    return resized


def list_files(base_path, valid_exts=valid_exts, contains=None):
    # Loop over the directory
    for (root_dir, dir_names, file_names) in os.walk(base_path):
        # Loop over the file names in the current directory
        for fn in file_names:
            if contains is not None and fn.find(contains) == -1:
                continue

            # Determine the file extension of the current file
            ext = fn[fn.rfind('.'):].lower()

            # Check to see if the file is an image and should be processed
            if ext.endswith(valid_exts):
                image_path = os.path.join(root_dir, fn).replace(' ', '\\ ')
                yield image_path


# List all images in a directory
def list_images(base_path, contains=None):
    return list_files(base_path, valid_exts=valid_exts, contains=contains)


def build_montages(image_list, image_shape, montage_shape):
    """
    ---------------------------------------------------------------------------------------------
    author: Kyle Hounslow
    ---------------------------------------------------------------------------------------------
    Converts a list of single images into a list of 'montage' images of specified rows and columns.
    A new montage image is started once rows and columns of montage image is filled.
    Empty space of incomplete montage images are filled with black pixels
    ---------------------------------------------------------------------------------------------
    :param image_list: python list of input images
    :param image_shape: tuple, size each image will be resized to for display (width, height)
    :param montage_shape: tuple, shape of image montage (width, height)
    :return: list of montage images in numpy array format
    ---------------------------------------------------------------------------------------------
    example usage:
    # load single image
    img = cv2.imread('lena.jpg')
    # duplicate image 25 times
    num_imgs = 25
    img_list = []
    for i in xrange(num_imgs):
        img_list.append(img)
    # convert image list into a montage of 256x256 images tiled in a 5x5 montage
    montages = make_montages_of_images(img_list, (256, 256), (5, 5))
    # iterate through montages and display
    for montage in montages:
        cv2.imshow('montage image', montage)
        cv2.waitKey(0)
    ----------------------------------------------------------------------------------------------
    """
    if len(image_shape) != 2:
        raise Exception('image shape must be list or tuple of length 2 (rows, cols)')

    if len(montage_shape) != 2:
        raise Exception('montage shape must be list or tuple of length 2 (rows, cols)')

    image_montages = []
    # start with black canvas to draw images onto
    montage_image = np.zeros(shape=(image_shape[1] * (montage_shape[1]), image_shape[0] * montage_shape[0], 3),
                             dtype=np.uint8)
    cursor_pos = [0, 0]
    start_new_img = False

    for img in image_list:
        if type(img).__module__ != np.__name__:
            raise Exception('input of type {} is not a valid numpy array'.format(type(img)))
        start_new_img = False
        img = cv2.resize(img, image_shape)
        # draw image to black canvas
        montage_image[cursor_pos[1]:cursor_pos[1] + image_shape[1], cursor_pos[0]:cursor_pos[0] + image_shape[0]] = img
        cursor_pos[0] += image_shape[0]  # increment cursor x position

        if cursor_pos[0] >= montage_shape[0] * image_shape[0]:
            cursor_pos[1] += image_shape[1]  # increment cursor y position
            cursor_pos[0] = 0
            if cursor_pos[1] >= montage_shape[1] * image_shape[1]:
                cursor_pos = [0, 0]
                image_montages.append(montage_image)
                # reset black canvas
                montage_image = np.zeros(
                    shape=(image_shape[1] * (montage_shape[1]), image_shape[0] * montage_shape[0], 3),
                    dtype=np.uint8)
                start_new_img = True

    if start_new_img is False:
        image_montages.append(montage_image)  # add unfinished montage

    return image_montages


def flip(image, random_flip):
    """
    Flip the image
    :param image: The image to flip
    :param random_flip:
    :return: The flipped image
    """

    if random_flip and np.random.choice([True, False]):
        image = np.fliplr(image)

    return image


def to_rgb(img):
    """
    Convert an image to color space RGB
    :param img: The image need to be converted
    :return: The converted image
    """

    width, height = img.shape
    ret = np.empty((width, height, 3), dtype=np.uint8)
    ret[:, :, 0] = ret[:, :, 1] = ret[:, :, 2] = img

    return ret


def crop(image, random_crop, image_size):
    """
    Crop the image with or w/o random
    :param image: The image need to be cropped
    :param random_crop: True or False
    :param image_size: The cropped image
    :return:
    """

    if image.shape[1] > image_size:
        sz1 = int(image.shape[1] // 2)
        sz2 = int(image_size // 2)
        if random_crop:
            diff = sz1 - sz2
            (h, v) = (np.random.randint(-diff, diff + 1),
                      np.random.randint(-diff, diff + 1))
        else:
            (h, v) = (0, 0)
        image = image[(sz1 - sz2 + v):(sz1 + sz2 + v),
                (sz1 - sz2 + h):(sz1 + sz2 + h), :]

    return image


In [2]:
from __future__ import division

import cv2 as cv
import math
import os
import numpy as np
import random
from scipy import ndimage
from matplotlib import pyplot as plt

from cv_utils.constants import *
from cv_utils import Box, utils


LABEL_COLORS = [COL_WHITE, COL_GREEN, COL_YELLOW, COL_MAGENTA]

_DEF_PLOT_OPTS = dict(cmap='jet', vmin=0, vmax=1)


def remove_bg(img, th=(240, 255)):
    """
    Removes similar colored background in the given image.
    :param img: Input image
    :param th: Tuple(2)
        Background color threshold (lower-limit, upper-limit)
    :return: Background removed image as result
    """
    if img.size == 0:
        return img

    img = gray3(img)

    # delete rows with complete background color
    h, w = img.shape[:2]
    i = 0
    while i < h:
        mask = np.logical_or(img[i, :, :] < th[0], img[i, :, :] > th[1])
        if not mask.any():
            img = np.delete(img, i, axis=0)
            i -= 1
            h -= 1
        i += 1

    # if image is complete background only
    if img.size == 0:
        return img

    # delete columns with complete background color
    h, w = img.shape[:2]
    i = 0
    while i < w:
        mask = np.logical_or(img[:, i, :] < th[0], img[:, i, :] > th[1])
        if not mask.any():
            img = np.delete(img, i, axis=1)
            i -= 1
            w -= 1
        i += 1

    return img


def add_bg(img, padding, color=COL_WHITE):
    """
    Adds a padding to the given image as background of specified color
    :param img: Input image.
    :param padding: constant padding around the image.
    :param color: background color that needs to filled for the newly padded region.
    :return: New image with background.
    """
    img = gray3(img)
    h, w, d = img.shape
    new_img = np.ones((h + 2*padding, w + 2*padding, d)) * color[:d]
    new_img = new_img.astype(np.uint8)
    set_img_box(new_img, (padding, padding, w, h), img)
    return new_img


def img_box(img, box):
    """
    Selects the sub-image inside the given box
    :param img: Image to crop from
    :param box: Box to crop from. Box can be either Box object or array of [x, y, width, height]
    :return: Cropped sub-image from the main image
    """
    if isinstance(box, tuple):
        box = Box.from_tup(box)

    if len(img.shape) == 3:
        return img[box.y:box.y + box.height, box.x:box.x + box.width, :]
    else:
        return img[box.y:box.y + box.height, box.x:box.x + box.width]


def set_img_box(img, box, value):
    """
    Updates the given value inside the specified box of the input image
    :param img: Input image
    :param box: Box dimension
    :param value: new image value
    :return: Update image
    """
    if isinstance(box, tuple):
        box = Box.from_tup(box)

    if len(img.shape) == 3:
        img[box.y:box.y + box.height, box.x:box.x + box.width, :] = value
    else:
        img[box.y:box.y + box.height, box.x:box.x + box.width] = value


def add_text_img(img, text, pos, box=None, color=None, thickness=1, scale=1, vertical=False):
    """
    Adds the given text in the image.
    :param img: Input image
    :param text: String text
    :param pos: (x, y) in the image or relative to the given Box object
    :param box: Box object. If not None, the text is placed inside the box.
    :param color: Color of the text.
    :param thickness: Thickness of the font.
    :param scale: Font size scale.
    :param vertical: If true, the text is displayed vertically. (slow)
    :return:
    """
    if color is None:
        color = COL_WHITE

    text = str(text)
    top_left = pos
    if box is not None:
        top_left = box.move(pos).to_int().top_left()
        if top_left[0] > img.shape[1]:
            return

    if vertical:
        if box is not None:
            h, w, d = box.height, box.width, 3
        else:
            h, w, d = img.shape
        txt_img = np.zeros((w, h, d), dtype=np.uint8)
        # 90 deg rotation
        top_left = h - pos[1], pos[0]
        cv.putText(txt_img, text, top_left, cv.FONT_HERSHEY_PLAIN, scale, color, thickness)

        txt_img = ndimage.rotate(txt_img, 90)
        mask = txt_img > 0
        if box is not None:
            im_box = img_box(img, box)
            im_box[mask] = txt_img[mask]
        else:
            img[mask] = txt_img[mask]
    else:
        cv.putText(img, text, top_left, cv.FONT_HERSHEY_PLAIN, scale, color, thickness)


def add_rect(img, box, color=None, thickness=1):
    """
    Draws a bounding box inside the image.
    :param img: Input image
    :param box: Box object that defines the bounding box.
    :param color: Color of the box
    :param thickness: Thickness of line
    :return: Rectangle added image
    """
    if color is None:
        color = COL_GRAY

    box = box.to_int()
    cv.rectangle(img, box.top_left(), box.bottom_right(), color, thickness)


def add_view_box(img, vbox):
    if vbox.color is None:
        vbox.color = COL_GRAY

    add_rect(img, vbox, vbox.color, vbox.thickness)

    for i, label in enumerate(vbox.labels):
        vertical = False
        if label.angle == 90:
            vertical = True
        add_text_img(img, label.text, label.pos, vbox, label.color, vertical=vertical)


def show_img(img, options=None):
    if is_gray(img):
        op = _DEF_PLOT_OPTS.copy()
        op.update(options)
        plt.imshow(img, cmap=op['cmap'], vmin=op['vmin'], vmax=op['vmax'])
    else:
        # convert from BGR to RGB before plotting
        plt.imshow(img[:, :, ::-1])


def imshow(*imgs, **options):
    """
    Plots multiple images using matplotlib
        by dynamically finding the required number of rows and cols.
    :param imgs: Images as any number of arguments
    :param options: Dict of options
        - cmap: Color map for gray scale images
        - vmin: Minimum value to be used in color map
        - vmax: Maximum value to be used in color map
    """
    n = len(imgs)
    nrows = int(math.ceil(math.sqrt(n)))
    ncols = int(math.ceil(n / nrows))
    for row in range(nrows):
        for col in range(ncols):
            i = row * ncols + col
            if i >= n:
                break
            plt.subplot(nrows, ncols, i+1)
            show_img(imgs[i], options)
    plt.show()


def collage(imgs, size, padding=10, bg=COL_BLACK):
    """
    Constructs a collage of same-sized images with specified padding.
    :param imgs: Array of images. Either 1d-array or 2d-array.
    :param size: (no. of rows, no. of cols)
    :param padding: Padding space between each image
    :param bg: Background color for the collage. Default: Black
    :return: New collage
    """
    # make 2d array
    if not isinstance(imgs[0], list):
        imgs = [imgs]

    h, w = imgs[0][0].shape[:2]
    nrows, ncols = size
    nr, nc = nrows * h + (nrows-1) * padding, ncols * w + (ncols-1) * padding

    res = np.ones((nr, nc, 3), dtype=np.uint8) * np.array(bg, dtype=np.uint8)

    for r in range(nrows):
        for c in range(ncols):
            img = imgs[r][c]

            if is_gray(img):
                img = gray3ch(img)

            rs = r * (h + padding)
            re = rs + h

            cs = c * (w + padding)
            ce = cs + w

            res[rs:re, cs:ce, :] = img

    return res


def repeat(img, size, padding=10, bg=COL_BLACK):
    imgs = []
    for _ in range(size[0]):
        img_row = []
        for _ in range(size[1]):
            img_row.append(img)
        imgs.append(img_row)
    return collage(imgs, size, padding, bg)


def is_gray(img):
    return len(img.shape) == 2


def gray3(img):
    return img[:, :, np.newaxis] if is_gray(img) else img


def gray3ch(img):
    return cv.cvtColor(img, cv.COLOR_GRAY2BGR) if is_gray(img) else img


def each_img(img_dir):
    """
    Reads and iterates through each image file in the given directory
    """
    for fname in utils.each_img(img_dir):
        fname = os.path.join(img_dir, fname)
        yield cv.imread(fname), fname


def resize_max(img, max_side):
    """
    Resize the image to threshold the maximum dimension within max_side
    :param img:
    :param max_side: Length of the maximum height or width
    :return:
    """
    h, w = img.shape[:2]
    if h > w:
        nh = max_side
        nw = w * (nh / h)
    else:
        nw = max_side
        nh = h * (nw / w)

    return cv.resize(img, (nw, nh))


def randomly_place(img, template):
    h, w = img.shape[:2]
    th, tw = template.shape[:2]
    rand_r, rand_c = random.randrange(h - th), random.randrange(w - tw)
    box = Box(rand_c, rand_r, tw, th)
    set_img_box(img, box, template)
    return box


def contrast(img, alpha):
    return cv.multiply(img, alpha)


def brightness(img, alpha):
    return cv.add(img, alpha)


def rot90(img):
    img = img.transpose((1,0,2))
    return cv.flip(img, 0)

ModuleNotFoundError: No module named 'scipy'